In [1]:
from pathlib import Path
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np, pandas as pd
import sys
sys.path.append("utils")
import ds_utils as DU
print("DU loaded from :", DU.__file__)
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_recall_fscore_support
from sklearn.manifold import TSNE
import cv2

DU loaded from : C:\Users\Pars\Desktop\my computer\taha\zohram\Ziraxa\project\ML\FarsiOCR\model\utils\ds_utils.py


In [2]:
repo = Path.cwd().parents[0]
tfm_tr = transforms.Compose([transforms.RandomRotation(10), transforms.RandomAffine(0, translate=(0.05, 0.05)), transforms.ToTensor()])
tfm_va = transforms.ToTensor()

class DS(Dataset):
    def __init__(self, df, root , tfm):
        self.df = df.reset_index(drop=True); self.root = root; self.tfm = tfm
    def __len__(self):
        return len(self.df)
    def __getitem__(self, i):
        r = self.df.iloc[i]; p = self.root/"Dataset/Create Dataset"/r["image_path"]
        img = Image.open(p).convert("L"); x = self.tfm(img)
        y = int(r.get("label_id", DU.label2idx[r["label"]]))
        return x, y

df_tr = pd.read_csv(repo/"Dataset/Create Dataset/train.csv")
df_va = pd.read_csv(repo/"Dataset/Create Dataset/val.csv")
tr = DS(df_tr, repo, tfm_tr); va = DS(df_va, repo, tfm_va)
tr_dl = DataLoader(tr, batch_size=64, shuffle=True); va_dl = DataLoader(va, batch_size=128)


In [3]:
class SmallCNN(nn.Module):
    def __init__(self, ncls=32):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1,32,3,padding = 1),nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,padding = 1),nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64,128,3,padding = 1),nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),nn.Linear(128*8*8, 256), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(256, ncls)
        )
    def forward(self, x):
        return self.net(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=SmallCNN().to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
crit = nn.CrossEntropyLoss()

In [9]:
def eval_dl(mdl, dl, device):
    mdl.eval(); ys = []; ps=[]
    with torch.no_grad():
        for x, y in dl:
            x = x.to(device)
            out = mdl(x)
            ys.append(y.cpu().numpy())
            ps.append(out.argmax(1).cpu().numpy())
    y = np.concatenate(ys); p = np.concatenate(ps)
    return DU.metrics(y, p)
best_f1 = 0; best = None
for epoch in range(1, 11):
    model.train()
    tot=0; corr = 0
    for x, y in tr_dl:
        x = x.to(device); y = y.to(device)
        opt.zero_grad(); out = model(x); loss=crit(out, y);loss.backward(); opt.step()
        tot+= y.size(0); corr += (out.argmax(1) == y).sum().item()
    acc_train = corr/ tot
    acc, f1m, *_ = eval_dl(model, va_dl, device)
    print(f"[EP{epoch}] train_acc={acc_train:.4f} val_acc={float(acc):.4f} val_f1m={float(f1m):.4f}")
    if f1m>best_f1: best_f1=f1m; best= {k:v.cpu().clone() for k,v in model.state_dict().items()}
if best: model.load_state_dict(best)


C:\Users\Pars\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Pars\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Pars\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

ValueError: could not convert string to float: 'acc'